## Script to load Synthea bundle and:

- Unbundle using dict and add meta profiles
- Save to USCore Validator and Validate
- Create a second transaction bundle of Provenance Resources for each Resource
- Load to FHIR Reference Server for further testing


for some reason the pyFHIRR4 models for Bundle not working :-(

- python version 3.6+
- get data from folder
- unbundle
- display


In [14]:
from os import listdir, path, unlink
from json import load, dumps, loads
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhirclient.r4models import bundle as B
from fhirclient.r4models import provenance as P
from fhirclient.r4models import narrative as N
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.fhirreference as FR
import fhirclient.r4models.meta as M
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from datetime import datetime, date
from requests import get, post, put
from uuid import uuid1
from time import sleep
from pprint import pprint

### globals

In [15]:

types = dict (
Patient = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
AllergyIntolerance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
CarePlan = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
CareTeam = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
Condition = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
Device = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device',
DiagnosticReport = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab',
                    'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note'],
DocumentReference = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
Encounter = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
Goal = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
Immunization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
Location = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
Medication = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
MedicationRequest = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
Observation = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab',
               'http://hl7.org/fhir/StructureDefinition/vitalsigns',
                'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age',
               'http://hl7.org/fhir/us/core/StructureDefinition/us-core-pulse-oximetry',],
Organization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
Practitioner = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
PractitionerRole = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
Procedure = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
Provenance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-provenance',
)


in_path = '/Users/ehaas/Documents/FHIR/synthea/output/fhir/'
in_path = '/Users/ehaas/Documents/FHIR/synthea/output/us_core_r4/fhir/'
#in_path='test/'
out_path = '/Users/ehaas/Documents/FHIR/ArgoR4Validator/source/examples/'  # append forward slash 
#f_name = 'Denisse335_Stracke611_e760e08d-a0ff-4cd7-83c0-36b27dae6ec3'
#f_name='Alaina222_Gleichner915_0e83bdc3-52c5-4e7e-ba92-174e711ba95c'
f_name = 'Nadia817_Morissette863_190017e3-417c-46d4-8c0e-dcc9dbc67d25.json'

f_now = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
f_now.as_json()

'2019-09-12T19:06:02.529282Z'

### Get files

In [16]:
def open_file(in_path,f_name): # get files
    with open(f'{in_path}{f_name}',encoding='utf-8', errors='ignore') as f:
        r = f.read()
       
        return(loads(r))

### First Clear the directory

In [17]:
def clear_path():
    for filename in listdir(out_path):
        file_path = path.join(out_path,filename)
        try:
            if path.isfile(file_path):
               unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(repr(e))

### Write file to examples dir in IG

In [18]:
def write_file(r_json,type,id): # write file  
    with open(f'{out_path}{type}-{id}.json', 'w') as f:
        #print(f'Writing to {out_path}{type}-{id}.json')
        f.write(r_json)

### Fetch Organization from the Bundle to create Provenance resource

In [19]:
def prefetch_org(b):
    #bundle = B.Bundle(b, strict=False)
    #prov_org = (i for i in bundle.entry if i.resource.resource_type =='Organization')
    orgs = (i for i in b['entry'] if i['resource']['resourceType'] =='Organization')
    org_id  = next(orgs)['resource']['id']
    return(org_id)

### Post Transaction Bundle to reference Server

In [20]:
def post_transaction(b):
    # note that bundle b is a dict not a class
    types = [i['resource']['resourceType'] for i in b['entry']]
    display(HTML(f'<h1>bundle</h1><h3>resources</h3> {types}'))
    fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json',
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    r = post(fhir_test_server, params = params, headers = headers, data = dumps(b, indent=4))
    display(HTML('<h1>Request Headers</h1>')) 
    pprint(r.request.headers)
    #display(HTML('<h1>Request Body</h1>')) 
    #pprint(r.request.body)

    display(HTML(f'<h1>Server Response</h1><h3>Status Code = {r.status_code}</h3>')) 
    pprint(r.headers)
    pprint(r.json())
            

### Create Provenance

- Inputs:
  - what resource is it about
  - Prefetched Organization to use for the author and transmitter ( for now just one will do)
- most everything else is a fixed value.
- validate using the argovalidator  (load to examples)
- add to bundle


In [21]:
def get_agent_type(system,code):
    
    prov_type = CC.CodeableConcept()
    prov_type_coding = C.Coding()
    prov_type_coding.system = system
    prov_type_coding.code = code
    prov_type.coding = [prov_type_coding]
    #print(prov_type)
    return(prov_type)

In [22]:
def get_prov(org,target_type, target_id):
    
  
    who_ref=FR.FHIRReference(dict(
                reference=f'Organization/{org}'
        ))
                             
    prov_target = FR.FHIRReference(dict(
            reference=f'{target_type}/{target_id}'
        ))
    
    f_now = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
    
    prov = P.Provenance()
    prov.id = str(uuid1())
    prov.meta = M.Meta(dict(profile =[types['Provenance']]))
    prov.target= [prov_target]
    prov.recorded = f_now
    prov.agent = []
    
    # this could be a function
    prov_agent = P.ProvenanceAgent()
    prov_agent.who = who_ref
    prov_agent.type = get_agent_type(system='http://hl7.org/fhir/us/core/CodeSystem/us-core-provenance-participant-type',code='transmitter')
    prov.agent.append(prov_agent)
    
    prov_agent = P.ProvenanceAgent()
    prov_agent.who = who_ref
    prov_agent.type = get_agent_type(system='http://terminology.hl7.org/CodeSystem/provenance-participant-type',code='author')
    prov.agent.append(prov_agent)
    
    #print(f'created resource Provenance-{prov.id} for resource {target_type}/{target_id}' )
    
    return(prov)    
    

### Loop through all entries and if US Core type change logical to literal reference and save

#### Loop through entries and create literal to logical reference map

In [23]:
def add_meta(r):
    
    global r_list
                          
    t = r["resourceType"]
    i = r["id"]
    if t in types.keys(): 
       r_list.append(t)
       if t not in ["DiagnosticReport",'Observation']: # add meta profiles
           r['meta']= (dict(profile =[types[t]]))
       elif "Observation" == t:  # "Observation"
           try:
               #['vital-signs','laboratory','72166-2']:
               if 'laboratory' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][0]])
               elif '72166-2' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][2]])
               elif '77606-2' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][3]])
               elif '59576-9' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][4]])
               elif '2708-6' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][5]])
               elif 'vital-signs' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][1]])
               else: # use lab
                   r['meta']= dict(profile =[types[t][0]])
                   pass
           except KeyError:
                   r['meta']= dict(profile =[types[t][0]])

       else:  # "DiagnosticReport"
           try:
               if 'LAB' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][0]])
               elif r['category'][0]['coding'][0]['code'] in ['LP29684-5','LP29708-2','LP7839-6',]:
                   r['meta']= dict(profile =[types[t][1]])
               else:  # use lab
                   r['meta']= dict(profile =[types[t][0]])
                   pass
           except KeyError:
               r['meta']= dict(profile =[types[t][0]])
                
    '''
    try:
        print(f' update resource {t}-{i} with meta {r["meta"]["profile"][0]}' )
    except Exception as e:
        print(repr(e))
    '''
              
    return(r)
  

In [24]:
### Validate Bundle entries using US Core Validator

In [25]:
def validate_me(bundles):
    for b in bundles:
        for e in b["entry"]:
                r = e["resource"]
                r_id = r["id"]
                #print(f'id = {r_id}')
                t = r["resourceType"]
                #print(f'type = {t}')
                # convert all logical to literal references
                r_string = dumps(r,indent=4)
                for ref_id, ref_type in ids.items(): 
                    r_string = r_string.replace(f'urn:uuid:{ref_id}',f'{ref_type}/{ref_id}')    
                # validate using the validation build

                write_file(r_string,t,r_id) # validate using the argovalidator (load to examples)
    display(Markdown('**All files loaded onto Validator - run IG-Pub using -d option to validate**'))

## Main File call all the functions above

In [26]:
clear_path()
r_list= []

synthea_files = [f_name] if f_name else [i for i in listdir(in_path)]

for file in synthea_files: # assume is synthea patient bundle.
    ids = {}
    provs = []
    if file.endswith(".json"):
        #print("json file found:\t", file)
        b = open_file(in_path,file)
        new_b = b
        org_id = prefetch_org(b) # for provenance
        #print(f'Provenance Organization url will be: Organization/{org_id}')
        for e in b["entry"]:
            r = e["resource"]
            r_id = r["id"]
            t = r["resourceType"]
            
            if t in types.keys(): # see what types are in bundle
               r_list.append(t)
            # get a list of id's to create literal references later using a string replace 
            ids[r_id]=t

            #e['resource']= add_meta(r) # add meta elements to resource  - Synthea already does this!!!
            
            #change POST to PUT and update the URL
            e['request']['method']='PUT'
            e['request']['url']=f'{e["request"]["url"]}/{r_id}'
            
            # create provenance and add to list
            provs.append(get_prov(org_id,t,r_id)) #create corresponding Provenance resource and add to list
            
        # create a transaction bundle provenance ,stick with dict type
        pb = dict(
            resourceType="Bundle",
            type="transaction",
            )
        
        pb['entry'] = []
        for p in provs:
            ids[p.id]='Provenance'
            
            e = dict( # no fullURL for now - use relative literal references
                resource = p.as_json(),
                request = dict(
                    method='PUT',
                    url= f'Provenance/{p.id}',
                    ), 
                )
            
            pb['entry'].append(e)
            
            
  
        #write bundle to temp file to test manually
        for i, bundle in enumerate([pb,b]):
            with open(f'test_bundle_{i}.json', 'w') as f:
                f.write(dumps(bundle, indent=4))
            
        # *********validate using the validation build ********
        
        validate_me([b,pb]) 
        
        
        #*********load bundles to FHIR Server - DOES NOT WORK - USE POSTMAN INSTEAD ***********
        '''
        for bundle in [b,pb]:
            r = post_transaction(bundle)
            sleep(15)
         '''   

print(f'number of example resource = {len(r_list)}')          
print(f'US Core types = {types.keys()}')
print(f'Synthea types {set(r_list)}')            
print(f'What is missing {set(types)-set(r_list)}')

print('======Fin======')

**All files loaded onto Validator - run IG-Pub using -d option to validate**

number of example resource = 361
US Core types = dict_keys(['Patient', 'AllergyIntolerance', 'CarePlan', 'CareTeam', 'Condition', 'Device', 'DiagnosticReport', 'DocumentReference', 'Encounter', 'Goal', 'Immunization', 'Location', 'Medication', 'MedicationRequest', 'Observation', 'Organization', 'Practitioner', 'PractitionerRole', 'Procedure', 'Provenance'])
Synthea types {'Observation', 'Practitioner', 'Location', 'Patient', 'MedicationRequest', 'Device', 'Goal', 'Encounter', 'DiagnosticReport', 'Immunization', 'Medication', 'Provenance', 'Procedure', 'AllergyIntolerance', 'CareTeam', 'PractitionerRole', 'CarePlan', 'Condition', 'Organization'}
What is missing {'DocumentReference'}
======Fin======
